In [4]:
from pathlib import Path
import sys

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd
import librosa
from scipy.io import wavfile

from Code.audio import AudioPreproc, PreprocCfg, AudioFeat, AudioFeatConfig

run = 1

input_dir_base = PROJECT_ROOT / "Database" / "data" / "audio"
output_dir_base = PROJECT_ROOT / "Database" / "tmp" / f"audioFeatTry{run:02d}"
output_dir_base.mkdir(parents=True, exist_ok=True)

labels = ["contar", "proporcion", "salir"]

pre_config = PreprocCfg(
    target_sr=16000, t_sec=1.2,
    frame_ms=25.0, hop_ms=10.0,
    highpass_hz=40.0, hp_order=2,
    preemph_a=0.97,
    vad_thresh_db=-35.0, vad_win_ms=20.0,
    vad_min_ms=120.0, vad_expand_ms=60.0,
    norm_mode="rms", rms_target_dbfs=-20.0,
    peak_ref=0.98, max_gain_db=18.0,
    gate_dbfs=-60.0, pad_mode="edge",
)
pre = AudioPreproc(pre_config)

feat_config = AudioFeatConfig(
    n_mfcc=13,
    delta_order=1,
    add_rms=True,
    add_zcr=True,
    stats=("mean", "std", "p10", "p90"),
)
feat = AudioFeat(pre=pre, cfg=feat_config)
feature_names = feat.nombres_de_caracteristicas()

y_proc, sr_proc, file_names, class_names = [], [], [], []
y_features = []

for cls in labels:
    input_dir = input_dir_base / cls
    output_dir = output_dir_base / cls
    output_dir.mkdir(parents=True, exist_ok=True)

    for audio_path in sorted(input_dir.glob("*.wav")):
        y, sr = pre.preprocesar_desde_path(audio_path)

        # guardar WAV procesado (float32 -> int16)
        y_int16 = np.clip(y * 32767.0, -32768, 32767).astype(np.int16)
        out_path = output_dir / audio_path.name
        wavfile.write(out_path, sr, y_int16)

        y_feat = feat.extract(y, pre_config.target_sr)

        y_proc.append(y)
        sr_proc.append(sr)
        file_names.append(audio_path.name)
        class_names.append(cls)
        y_features.append(y_feat)

# Tabla de estadísticos básicos por archivo (opcional)
stats_rows = []
for cls, fname, y, sr in zip(class_names, file_names, y_proc, sr_proc):
    stats_rows.append({
        "Clase": cls,
        "Archivo": fname,
        "Duración (s)": len(y) / sr,
        "Mín": float(np.min(y)),
        "Máx": float(np.max(y)),
        "Energía RMS": float(np.sqrt(np.mean(y**2))),
    })
df_stats = pd.DataFrame(stats_rows)
df_stats.to_csv(output_dir_base / "estadisticos.csv", index=False)

# Tabla de features completas por archivo
feat_rows = []
for cls, fname, vec in zip(class_names, file_names, y_features):
    row = {"Clase": cls, "Archivo": fname}
    row.update({name: float(val) for name, val in zip(feature_names, vec)})
    feat_rows.append(row)

df_feat = pd.DataFrame(feat_rows)
df_feat.to_csv(output_dir_base / "features.csv", index=False)

df_feat  # muestra la tabla de features en el notebook


19200


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 117 and the array at index 1 has size 9